# Allow for image level parameter input: added to *digipath_toolkit.py*

## OpenSlide: Bug == hard to read documentation *read_location*

[github issue 222](https://github.com/openslide/openslide/issues/222) <br>
[StackOverflow ? is there a problem ?](https://stackoverflow.com/questions/56785530/is-there-any-problem-with-the-openslide-read-region-function) <br>

## *image_file_to_patches_directory_for_image_level.py*

In [ ]:
import time
nb_start_time = time.time()

import os
import sys

from collections import OrderedDict
import argparse

import numpy as np
import pandas as pd
import yaml

from skimage.filters import threshold_otsu
from skimage.color import rgb2lab

from PIL import ImageDraw
from PIL import TiffImagePlugin as tip

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

COMMON_THUMBNAIL_DIVISOR = 20

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [1]:
def image_file_to_patches_directory_for_image_level_NB(run_parameters):
    """ Usage: number_images_found = image_file_to_patches_directory_for_image_level(run_parameters)

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                output_dir:             writeable directory for the tfrecord
                                class_label:            label for all images
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                file_ext:               default is '.jpg' ('.png') was also tested
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                threshold:              minimimum sum of thresholded image (default = 0)

    Returns:                    None - writes images to output_dir (possibly many)
                                (prints number_images_found after all else)

    """
    image_file_name = run_parameters['wsi_filename']
    output_dir = run_parameters['output_dir']
    class_label = run_parameters['class_label']
    patch_width = run_parameters['patch_width']
    patch_height = run_parameters['patch_height']
    patch_size = (patch_width, patch_height)

    image_level = run_parameters['image_level']

    if 'file_ext' in run_parameters:
        file_ext = run_parameters['file_ext']
        if file_ext[0] != '.':
            file_ext = '.' + file_ext
    else:
        file_ext = '.jpg'

    if os.path.isdir(output_dir) == False:
        print('creating output directory:\n%s\n' % (output_dir))
        os.makedirs(output_dir)

    if len(file_ext) == 0:
        file_ext = '.jpg'
    elif file_ext[0] != '.':
        file_ext = '.' + file_ext

    _, file_name_base = os.path.split(image_file_name)
    file_name_base, _ = os.path.splitext(file_name_base)

    level_sizes_dict = get_level_sizes_dict(run_parameters['wsi_filename'])
    size_multiplier = level_sizes_dict['level_downsamples'][image_level]

    patch_location_array = get_patch_location_array_for_image_level(run_parameters)
    patch_location_array = [(int(p[0] * size_multiplier), int(p[1] * size_multiplier)) for p in patch_location_array]

    number_images_found = len(patch_location_array)
    patch_image_name_dict = {'case_id': file_name_base, 'class_label': class_label, 'file_ext': file_ext}

    # get the OpenSlide object - open the file, and get the mask with the scaled grids
    os_obj = openslide.OpenSlide(image_file_name)

    # iterate the list of locations found
    for read_location in patch_location_array:
        # build the patch name
        patch_image_name_dict['location_x'] = read_location[1]
        patch_image_name_dict['location_y'] = read_location[0]
        patch_name = dict_to_patch_name(patch_image_name_dict)
        patch_full_name = os.path.join(output_dir, patch_name)
        location = (read_location[1], read_location[0])

        # OpenSlide extract, convert, save
        patch_image = os_obj.read_region(location=location, level=image_level, size=patch_size)
        patch_image = patch_image.convert('RGB')
        patch_image.save(patch_full_name)

    os_obj.close()
    print('%i images found' % (number_images_found))


In [ ]:
data_dir = '../../DigiPath_MLTK_data/Aperio'
image_file_name = 'CMU-1.svs'

run_parameters = dict()

run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = COMMON_THUMBNAIL_DIVISOR
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0
run_parameters['image_level'] = 0

run_parameters['class_label'] = 'class_label_test_str'
run_parameters['output_dir'] = '../../run_dir/results'
run_parameters['file_ext'] = 'jpg' 

for k, v in run_parameters.items():
    print('%25s: %s'%(k,v))
    
lineprint_level_sizes_dict(run_parameters['wsi_filename'])

patch_location_array = image_file_to_patches_directory_for_image_level(run_parameters)
print(len(patch_location_array), 'patches')

t0 = time.time()
image_file_to_patches_directory_for_image_level(run_parameters)
tt = time.time() - t0

print('running time: %0.3f'%(tt))